### Autodock-SS

#### Connecting to SmartRedis and getting the docking Results

In [1]:
import numpy as np
from smartredis import Client

client = Client(address="127.0.0.1:6780", cluster=False)

# poll until data is available
client.poll_key("1stp_ligand.pdbqt_testing", 100, 100)

print("Dataset written")
dataset=client.get_dataset("1stp_ligand.pdbqt_testing")
print(dataset)

Dataset written


#### Getting some metadata and the input ligand

In [2]:
inputLigand = dataset.get_tensor("input_ligand")
print(inputLigand)

[82 69 77 ... 32 53 10]


#### SmartRedis store tensors, however it is possible to store and retrive strings, in our case, we are using Autodock's native data representation based on pdbqt format:

In [3]:
inputLigadStr = "".join([chr(char) for char in inputLigand])
print (inputLigadStr)

REMARK  5 active torsions:
REMARK  status: ('A' for Active; 'I' for Inactive)
REMARK    1  A    between atoms: C11_1  and  C10_4 
REMARK    2  A    between atoms: C10_4  and  C9_5 
REMARK    3  A    between atoms: C9_5  and  C8_6 
REMARK    4  A    between atoms: C8_6  and  C7_7 
REMARK    5  A    between atoms: C7_7  and  C2_8 
ROOT
HETATM    1  C11 BTN A 300       9.051   4.576 -14.459  1.00 21.57     0.172 C 
HETATM    2  O11 BTN A 300       8.147   5.373 -14.172  1.00 21.65    -0.648 OA
HETATM    3  O12 BTN A 300       8.697   3.498 -15.132  1.00 22.67    -0.648 OA
ENDROOT
BRANCH   1   4
HETATM    4  C10 BTN A 300      10.321   4.413 -13.702  1.00 20.77     0.113 C 
BRANCH   4   5
HETATM    5  C9  BTN A 300      10.215   3.247 -12.658  1.00 20.09     0.010 C 
BRANCH   5   6
HETATM    6  C8  BTN A 300      11.487   2.972 -11.964  1.00 19.24     0.002 C 
BRANCH   6   7
HETATM    7  C7  BTN A 300      11.627   1.558 -11.402  1.00 18.33     0.017 C 
BRANCH   7   8
HETATM    8  C2  BTN 

### Let's get some meta data

In [4]:
runs=dataset.get_meta_scalars("runs")
print("Number of runs: ", runs)
print("Number of energy evaluations: ", dataset.get_meta_scalars("nrg_evaluations"))
print("Number of generations: ", dataset.get_meta_scalars("generations"))
print("Size of population: ", dataset.get_meta_scalars("population_size"))

Number of runs:  [100]
Number of energy evaluations:  [244898]
Number of generations:  [42000]
Size of population:  [150]


### Let's visualize the Input Ligand with NGL, first we need to convert pdbqt string format to pdb format which is supported by NGL:

In [5]:
from openbabel import openbabel as ob

obc = ob.OBConversion()
obc.SetInAndOutFormats("pdbqt", "pdb")
mol = ob.OBMol()
obc.ReadString(mol,inputLigadStr)
inputLigadStr_pdb=obc.WriteString(mol)
print (inputLigadStr_pdb)

REMARK  5 active torsions:
REMARK  status: ('A' for Active; 'I' for Inactive)
REMARK    1  A    between atoms: C11_1  and  C10_4 
REMARK    2  A    between atoms: C10_4  and  C9_5 
REMARK    3  A    between atoms: C9_5  and  C8_6 
REMARK    4  A    between atoms: C8_6  and  C7_7 
REMARK    5  A    between atoms: C7_7  and  C2_8 
COMPND    UNNAMED
AUTHOR    GENERATED BY OPEN BABEL 3.1.0
HETATM    1  C11 BTN A 300       9.051   4.576 -14.459  1.00  0.00           C  
HETATM    2  O11 BTN A 300       8.147   5.373 -14.172  1.00  0.00           O  
HETATM    3  O12 BTN A 300       8.697   3.498 -15.132  1.00  0.00           O  
HETATM    4  C10 BTN A 300      10.321   4.413 -13.702  1.00  0.00           C  
HETATM    5  C9  BTN A 300      10.215   3.247 -12.658  1.00  0.00           C  
HETATM    6  C8  BTN A 300      11.487   2.972 -11.964  1.00  0.00           C  
HETATM    7  C7  BTN A 300      11.627   1.558 -11.402  1.00  0.00           C  
HETATM    8  C2  BTN A 300      10.923   1.0

### now visualize the Input Ligand in pdb format

In [6]:
import nglview

view = nglview.NGLWidget()

model = nglview.TextStructure(inputLigadStr_pdb,ext='pdb')
view = nglview.NGLWidget()
view.add_structure(model)

view.display(gui=True)


ThemeManager()

NGLWidget(gui_style='ngl')

In [7]:
#view[0].show()
#view[0].hide()